In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from tqdm.auto import tqdm

import optuna
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms

from Dataset_Construction import Balance_Ratio 
from Sampling import label_divide
from AdaClassifier import train_set, multiple_set, print_badC, bad_plot, line_chart, cf_matrix, runall_AdaBoostC
from AdaRegressor import AUC, PR_curve, multiple_curve, PR_matrix, best_threshold, runall_AdaBoostR
from Aging_Score import score1
from XGBoost import optuna_history

os.chdir('C:/Users/user/Desktop/Darui_R08621110')  
os.getcwd()

'C:\\Users\\user\\Desktop\\Darui_R08621110'

In [2]:
##### GPU ??? #####
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


### Dataloader

In [3]:
class RunhistSet(Dataset):
    
    def __init__(self, train_x, train_y):
        self.x = torch.tensor(train_x.values.astype(np.float32))
        self.y = torch.tensor(train_y.values.astype(np.float32)).long()
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return [self.x[idx], self.y[idx]]

### Neural Network

In [4]:
class NeuralNetwork(nn.Module):
    
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.stack = nn.Sequential(
            nn.Linear(140, 64),
            nn.Sigmoid(),
            nn.Dropout(0.25),
            nn.Linear(64, 64),
            nn.Sigmoid(),
            nn.Dropout(0.25),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        logits = self.stack(x)
        return logits

### training

In [5]:
def training(network, trainloader, validloader, optimizer, criterion, epoch, filename):
    
    network.train()
    best_model = network
    test_accurracy = 0
    
    for i in tqdm(range(epoch)):
        total_loss = 0
        accuracy = 0
        count = 0
        
        for x, y in trainloader:
            
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = network(x)
            loss = criterion(output, y)
            
            _, predicted = torch.max(output.data, 1)
            count += len(x)
            accuracy += (predicted == y).sum().item()
            total_loss += loss.item()*len(y)            
            
            loss.backward()
            optimizer.step()
            
        print(f"Train Loss: {total_loss / count}")
        print(f"Train Accuracy: {accuracy / count}")
        
    if (i % 10 == 0):
        valid_accuracy = test(network, validloader, criterion)
        if (valid_accuracy > test_accuracy):
            test_accuracy = valid_accuracy
            best_model = network
            
    torch.save(best_model, f'{filename}_nn_{epoch}.pt')
            
    return network

### testing

In [30]:
def test(network, dataloader, criterion):
    
    network.eval()
    total_loss = 0
    accuracy = 0
    count = 0
    
    for x, y in dataloader:
        
        x = x.to(device)
        y = y.to(device)
        output = network(x)
        loss = criterion(output, y)
        
        _, predicted = torch.max(output.data, 1)
        count += len(x)
        accuracy += (predicted == y).sum().item()
        total_loss += loss.item()*len(y)
        
    print(f"Test Loss: {total_loss / count}")
    print(f"Test Accuracy: {accuracy / count}")
    
    return (accuracy / count)

In [25]:
train_data = RunhistSet(trainset_x['set8'], trainset_y['set8'])
test_data = RunhistSet(run_test_x, run_test_y)
train_ratio = 0.75
train_size = int(len(train_data)*train_ratio)
valid_size = len(train_data) - train_size
train_data, valid_data = random_split(train_data, [train_size, valid_size])

train_loader = DataLoader(train_data, batch_size = 64, shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = 64, shuffle = False)
test_loader = DataLoader(test_data, batch_size = 64, shuffle = False)

In [26]:
model = NeuralNetwork().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 0.01)
criterion = nn.CrossEntropyLoss().to(device)

In [32]:
done_model = training(model, train_loader, valid_loader, optimizer, criterion, 1000, 'tamama')

Train Loss: 0.2506269671415028
Train Accuracy: 0.915296052631579
Train Loss: 0.24531482630654386
Train Accuracy: 0.9004934210526315
Train Loss: 0.23933225870132446
Train Accuracy: 0.9144736842105263
Train Loss: 0.24225999728629463
Train Accuracy: 0.9087171052631579
Train Loss: 0.23955487028548592
Train Accuracy: 0.9161184210526315
Train Loss: 0.24146685396370135
Train Accuracy: 0.9144736842105263
Train Loss: 0.24479138498243533
Train Accuracy: 0.9087171052631579
Train Loss: 0.23859280583105588
Train Accuracy: 0.9169407894736842
Train Loss: 0.23508757431256144
Train Accuracy: 0.9120065789473685
Train Loss: 0.22687146459755145
Train Accuracy: 0.921875
Train Loss: 0.2500513305789546
Train Accuracy: 0.9185855263157895
Train Loss: 0.24596371697752098
Train Accuracy: 0.9169407894736842
Train Loss: 0.2394544646928185
Train Accuracy: 0.9136513157894737
Train Loss: 0.24131895837030912
Train Accuracy: 0.912828947368421
Train Loss: 0.2384407528136906
Train Accuracy: 0.912828947368421
Train Loss: 

Train Loss: 0.24214119268091103
Train Accuracy: 0.9169407894736842
Train Loss: 0.23273437038848274
Train Accuracy: 0.9013157894736842
Train Loss: 0.22757942896140249
Train Accuracy: 0.921875
Train Loss: 0.24737641293751567
Train Accuracy: 0.9046052631578947
Train Loss: 0.24684874556566538
Train Accuracy: 0.9046052631578947
Train Loss: 0.24491263141757563
Train Accuracy: 0.912828947368421
Train Loss: 0.22996312222982707
Train Accuracy: 0.9251644736842105
Train Loss: 0.22012527600715034
Train Accuracy: 0.9276315789473685
Train Loss: 0.23064741178562767
Train Accuracy: 0.921875
Train Loss: 0.25309381202647563
Train Accuracy: 0.9004934210526315
Train Loss: 0.22475264260643407
Train Accuracy: 0.915296052631579
Train Loss: 0.2440964461941468
Train Accuracy: 0.9202302631578947
Train Loss: 0.25806316027515813
Train Accuracy: 0.9111842105263158
Train Loss: 0.23301841004898674
Train Accuracy: 0.9144736842105263
Train Loss: 0.25036481414970596
Train Accuracy: 0.90625
Train Loss: 0.224692657589912

Train Loss: 0.24579658320075587
Train Accuracy: 0.9235197368421053
Train Loss: 0.21675748103543332
Train Accuracy: 0.9300986842105263
Train Loss: 0.2452928204285471
Train Accuracy: 0.9120065789473685
Train Loss: 0.25100813806056976
Train Accuracy: 0.912828947368421
Train Loss: 0.235875782213713
Train Accuracy: 0.9243421052631579
Train Loss: 0.2327794158145001
Train Accuracy: 0.921875
Train Loss: 0.22906229841081718
Train Accuracy: 0.9226973684210527
Train Loss: 0.23828901977915512
Train Accuracy: 0.915296052631579
Train Loss: 0.2152581152163054
Train Accuracy: 0.9268092105263158
Train Loss: 0.2483622768991872
Train Accuracy: 0.912828947368421
Train Loss: 0.2399599928604929
Train Accuracy: 0.9177631578947368
Train Loss: 0.240727142283791
Train Accuracy: 0.921875
Train Loss: 0.23260599845334104
Train Accuracy: 0.9243421052631579
Train Loss: 0.23018635103577062
Train Accuracy: 0.9243421052631579
Train Loss: 0.23527883620638596
Train Accuracy: 0.912828947368421
Train Loss: 0.24858822869627

Train Loss: 0.22178818912882553
Train Accuracy: 0.9169407894736842
Train Loss: 0.24018201310383647
Train Accuracy: 0.9185855263157895
Train Loss: 0.24475603825167605
Train Accuracy: 0.8988486842105263
Train Loss: 0.22782867048916064
Train Accuracy: 0.9177631578947368
Train Loss: 0.23475066611641332
Train Accuracy: 0.9169407894736842
Train Loss: 0.2225524265515177
Train Accuracy: 0.921875
Train Loss: 0.23014256122865176
Train Accuracy: 0.9161184210526315
Train Loss: 0.23887990571950612
Train Accuracy: 0.9120065789473685
Train Loss: 0.2493625306769421
Train Accuracy: 0.9078947368421053
Train Loss: 0.22647243816601603
Train Accuracy: 0.921875
Train Loss: 0.24208275111098038
Train Accuracy: 0.9095394736842105
Train Loss: 0.2219814509153366
Train Accuracy: 0.9333881578947368
Train Loss: 0.24284870138293818
Train Accuracy: 0.915296052631579
Train Loss: 0.2439854545028586
Train Accuracy: 0.9120065789473685
Train Loss: 0.2405968935866105
Train Accuracy: 0.9111842105263158
Train Loss: 0.2371714

Train Loss: 0.24175293194620232
Train Accuracy: 0.9161184210526315
Train Loss: 0.2347070010084855
Train Accuracy: 0.921875
Train Loss: 0.25181238274825246
Train Accuracy: 0.9095394736842105
Train Loss: 0.240390608185216
Train Accuracy: 0.9103618421052632
Train Loss: 0.2193368696852734
Train Accuracy: 0.9259868421052632
Train Loss: 0.2384280413389206
Train Accuracy: 0.9120065789473685
Train Loss: 0.2262353191250249
Train Accuracy: 0.9210526315789473
Train Loss: 0.24289927435548683
Train Accuracy: 0.9013157894736842
Train Loss: 0.2354908985526938
Train Accuracy: 0.9194078947368421
Train Loss: 0.24358533244383962
Train Accuracy: 0.9161184210526315
Train Loss: 0.22337763560445686
Train Accuracy: 0.9210526315789473
Train Loss: 0.23824198936161242
Train Accuracy: 0.9185855263157895
Train Loss: 0.2379238064351835
Train Accuracy: 0.9243421052631579
Train Loss: 0.22403371843852496
Train Accuracy: 0.9276315789473685
Train Loss: 0.24037275267274757
Train Accuracy: 0.9095394736842105
Train Loss: 0

Train Loss: 0.23587945731062637
Train Accuracy: 0.9120065789473685
Train Loss: 0.24418345877998754
Train Accuracy: 0.9177631578947368
Train Loss: 0.2374547092538131
Train Accuracy: 0.9177631578947368
Train Loss: 0.2321858060987372
Train Accuracy: 0.921875
Train Loss: 0.2448091569699739
Train Accuracy: 0.912828947368421
Train Loss: 0.2485617846250534
Train Accuracy: 0.915296052631579
Train Loss: 0.22645162987081627
Train Accuracy: 0.9251644736842105
Train Loss: 0.22479864092249618
Train Accuracy: 0.9194078947368421
Train Loss: 0.2366291559056232
Train Accuracy: 0.9185855263157895
Train Loss: 0.22569368701232106
Train Accuracy: 0.9317434210526315
Train Loss: 0.21341012731978767
Train Accuracy: 0.9226973684210527
Train Loss: 0.240995580428525
Train Accuracy: 0.912828947368421
Train Loss: 0.22536851700983548
Train Accuracy: 0.9268092105263158
Train Loss: 0.2519060937981856
Train Accuracy: 0.90625
Train Loss: 0.233895406126976
Train Accuracy: 0.9226973684210527
Train Loss: 0.237449125239723

Train Loss: 0.2353775713004564
Train Accuracy: 0.9226973684210527
Train Loss: 0.2499851184455972
Train Accuracy: 0.9078947368421053
Train Loss: 0.24634369345087753
Train Accuracy: 0.9078947368421053
Train Loss: 0.23403183880605197
Train Accuracy: 0.9177631578947368
Train Loss: 0.23889219760894775
Train Accuracy: 0.9037828947368421
Train Loss: 0.22863340456234782
Train Accuracy: 0.915296052631579
Train Loss: 0.2278597370574349
Train Accuracy: 0.9210526315789473
Train Loss: 0.23598741857629074
Train Accuracy: 0.9177631578947368
Train Loss: 0.24730783387234337
Train Accuracy: 0.9054276315789473
Train Loss: 0.2212115588941072
Train Accuracy: 0.9243421052631579
Train Loss: 0.22584033482953123
Train Accuracy: 0.9235197368421053
Train Loss: 0.24574684155614754
Train Accuracy: 0.9111842105263158
Train Loss: 0.22207958133597122
Train Accuracy: 0.9300986842105263
Train Loss: 0.2363875610263724
Train Accuracy: 0.9111842105263158
Train Loss: 0.24192929189456136
Train Accuracy: 0.9111842105263158
T

Train Loss: 0.23884249753073641
Train Accuracy: 0.9144736842105263
Train Loss: 0.23439231750212217
Train Accuracy: 0.9169407894736842
Train Loss: 0.22351528782593577
Train Accuracy: 0.9276315789473685
Train Loss: 0.23344588201296956
Train Accuracy: 0.921875
Train Loss: 0.22518673460734517
Train Accuracy: 0.9169407894736842
Train Loss: 0.24109360026685814
Train Accuracy: 0.9177631578947368
Train Loss: 0.23780010483766856
Train Accuracy: 0.9226973684210527
Train Loss: 0.2449321935051366
Train Accuracy: 0.9070723684210527
Train Loss: 0.22682222645533712
Train Accuracy: 0.9161184210526315
Train Loss: 0.24065864243005453
Train Accuracy: 0.9169407894736842
Train Loss: 0.24189535018644834
Train Accuracy: 0.912828947368421
Train Loss: 0.2530649610255894
Train Accuracy: 0.9070723684210527
Train Loss: 0.23798285503136485
Train Accuracy: 0.915296052631579
Train Loss: 0.2342883803342518
Train Accuracy: 0.9185855263157895
Train Loss: 0.23936871713713595
Train Accuracy: 0.9136513157894737
Train Loss

In [33]:
test(done_model, test_loader, criterion)

Test Loss: 1.1149038069993804
Test Accuracy: 0.44519643792561553


0.44519643792561553

## Data Processing

### runhist data

In [8]:
###multiple dataset###
data_dict = multiple_set(num_set = 10)
trainset_x, trainset_y = train_set(data_dict, num_set = 10, label = 'GB')

#####for runhist dataset#####
run_test = pd.read_csv('test_runhist.csv').iloc[:, 2:]
run_test_x, run_test_y = label_divide(run_test, None, 'GB', train_only = True)
print('\n', 'Dimension of run test:', run_test.shape)

Dimension of dataset 0 : (80518, 141)  balance ratio: 1101.9863
Dimension of dataset 1 : (1634, 141)  balance ratio: 1.0
Dimension of dataset 2 : (1498, 141)  balance ratio: 1.0
Dimension of dataset 3 : (1752, 141)  balance ratio: 1.0
Dimension of dataset 4 : (1608, 141)  balance ratio: 1.0
Dimension of dataset 5 : (1618, 141)  balance ratio: 1.00496
Dimension of dataset 6 : (1558, 141)  balance ratio: 1.08568
Dimension of dataset 7 : (1622, 141)  balance ratio: 1.0
Dimension of dataset 8 : (1622, 141)  balance ratio: 1.0
Dimension of dataset 9 : (803, 141)  balance ratio: 10.0

 10 datasets are loaded.

Labels of  10 datasets are divided.

 Dimension of run test: (47725, 141)
